# Lab 10: Working with FITS files
Based on the astropy tutorial http://docs.astropy.org/en/stable/io/fits/

Name:

## FITS Files
FITS (Flexible Image Transport System) is a portable file standard widely used in the astronomy community to store images and tables. A fits file generally contains two major pieces of information the **Header** and **Data/Table**.

## Loading a FITS File

FITS files can store more than one image/table and header. An HDU (Header Data Unit) is the highest level component of the FITS file structure, consisting of a header and (typically) a data array or table. When you open a fits file, you get a list of HDUs. YOu can see information about each HDU by using .info()

In [ ]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
# change some default plotting parameters
import matplotlib as mpl
mpl.rcParams['image.origin'] = 'lower'
mpl.rcParams['image.cmap'] = 'Greys_r'
%matplotlib inline

In [ ]:
hdulist = fits.open('data/aa_aql0007_raw.fits')
hdulist.info()
#Use hdulist.close() when you are done with the file

## FITS Header
If you are unfamiliar with the FITS file you can always look at the header. Header information is stored in the .header attribute for each HDU in your list.

In [ ]:
prihdr = hdulist[0].header
print(repr(prihdr))

The Header is filled with keyword value comment groups. You can get a list of the keys, and pick out specific keys if you want. You can reference them by name or by location.

In [ ]:
print(list(prihdr.keys()))
print("My Object: {}".format(prihdr['OBJECT']))
print("My Object: {}".format(prihdr[25]))

## Adding to or Updating a Header.
Headers are stored as dictionaries, so you can create a new value or update a value just by an assignment operation.  You can update the value and the comment at the same time by using a tuple.


Be careful about **Comment or History keywords**. There tend to be multiple of them in headers, so you have to reference them by location. Also, when you assign a comment or history it automatically creates a new comment block.

In [ ]:
prihdr['Num_Monk'] = (20,"Fun Times!")
print(list(prihdr.keys()))


## Now its your turn
Convert the RA and DEC for this object into degrees, and create two new keys ra_deg and dec_deg.

In [ ]:
## Accessing Image Data
image_data = hdulist[0].data

## Displaying Data
There are two ways to display an image. The first is using the command line tool Ds9. At the command prompt type ds9 imagename. Ds9 is a powerful tool for analyzing image data. It can handle multiple hdu and has a useful interface. 

You can also view the image in python.

In [ ]:
plt.figure()
plt.imshow(image_data)
plt.colorbar()

We need to fix the min and max values to something more reasonable.

In [ ]:
from astropy.visualization import ZScaleInterval
interval = ZScaleInterval()
(imin,imax) = interval.get_limits(image_data)
plt.imshow(image_data, vmin=imin,vmax=imax)
plt.colorbar()

## Getting Values
The image arrays are stored an 2d numpy arrays, so they can be manipulated in the same way. **Be careful**, the x and y in astropy image data are reversed compared to ds9! Also, ds9 starts at pixel [1,1] and astropy starts at [0,0]. I can cut-out regions using [miny:maxy,minx:maxx].

In [ ]:
small_image = image_data[0:130,0:220]
print(small_image.shape)
(imin,imax) = interval.get_limits(small_image)
plt.imshow(small_image, vmin=imin,vmax=imax)
plt.colorbar()

In [ ]:
print(image_data[61,60])

Your turn. What pixel in ds9 has the same value as 61,60?

Answer:

## Image Correction
Last week we discussed processing an image so that it can be used for photometry.

In [ ]:
hdulist.close() #Clean things up.
hdulist = fits.open('data/aa_aql0007_raw.fits')
image_data = hdulist[0].data
image_hdr = hdulist[0].header

## Bias Subtraction
The first step is to remove a bias from our image. In some cases, bias frames are called zeroes.

In [ ]:
bias = fits.open('data/Zero.fits')
bias_data = bias[0].data
step1 = image_data - bias_data
print(image_data[61,60])
print(step1[61,60])
image_hdr['comment'] = "Subtracted Bias"

## Dark Subtraction
Step 2 is Dark Subtraction (Note each Calibration image has already had the previous step applied to it).

In [ ]:
dark = fits.open('data/Dark.fits')
dark_hdr = dark[0].header
dark_data = dark[0].data
step2 = step1 - dark_data
image_hdr['comment'] = "Subtracted Dark"
print(image_data[61,60])
print(step1[61,60])
print(step2[61,60]) #Uh-Oh something is wrong?

In [ ]:
print(image_hdr['exptime'])
print(dark_hdr['exptime'])

In [ ]:
step2 = step1 - (dark_data *(image_hdr['exptime']/dark_hdr['exptime']))
print(image_data[61,60])
print(step1[61,60])
print(step2[61,60]) #Much Better

## Flat Fielding
Step 3 is to Flat Field the image. This is a normalization so we are dividing instead of subtracting.

In [ ]:
flat = fits.open('data/FlatV.fits')
flat_hdr = flat[0].header
flat_data = flat[0].data
print(np.average(flat_data)) #Is the average level of our Flat
step3 = step2 / flat_data
image_hdr['comment'] = "Divided Flat"
print(image_data[61,60])
print(step1[61,60])
print(step2[61,60]) #Much Better
print(step3[61,60]) #Hmmm this is a little strange

## Save your results
You can save your results by giving the function fits.writeto() your data and an optional header.

In [ ]:
fits.writeto('data/out.fits', step3, image_hdr,overwrite=True)

## Let's Compare against a formally processed image
In /data there is a formally reduced image called aa_aql0007.fits. Compare it to your image. How good a job did we do?
* Open both images in ds9
* Show the header of out.fits
* How are the min, max, median different
* Plot both images
* Try dividing the images
* Can you correct your image so that it has the same values as the processed image?
